In [1]:
from sampler import ImbalancedDatasetSampler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os

In [2]:
trainImgFolder='data/bihar/bihar_2010_landsat7_cutFiles_rgb_BF_train/'
testImgFolder='data/bihar/bihar_2010_landsat7_cutFiles_rgb_BF_test/'
checkPtFolder=trainImgFolder[:-1]+'_checkpoints'
os.makedirs(checkPtFolder, exist_ok=True)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 30
num_classes = 3
batch_size = 100
learning_rate = 0.001

myTransform = transforms.Compose(
                   [transforms.Resize((64,64)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.65, 0.65, 0.65), (0.15, 0.15, 0.15))])


train_dataset = torchvision.datasets.ImageFolder(root=trainImgFolder,
                                                transform=myTransform)

test_dataset = torchvision.datasets.ImageFolder(root=testImgFolder,
                                               transform=myTransform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           sampler=ImbalancedDatasetSampler(train_dataset))

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [3]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(16*16*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    file_name=checkPtFolder+'/'+'epoch_'+str(epoch)+'.ckpt'
    torch.save(model.state_dict(), file_name)

Epoch [1/30], Step [100/315], Loss: 0.9840
Epoch [1/30], Step [200/315], Loss: 1.0348
Epoch [1/30], Step [300/315], Loss: 0.7885
Epoch [2/30], Step [100/315], Loss: 0.6728
Epoch [2/30], Step [200/315], Loss: 0.5204
Epoch [2/30], Step [300/315], Loss: 0.4625
Epoch [3/30], Step [100/315], Loss: 0.5297
Epoch [3/30], Step [200/315], Loss: 0.4521
Epoch [3/30], Step [300/315], Loss: 0.3258
Epoch [4/30], Step [100/315], Loss: 0.3448
Epoch [4/30], Step [200/315], Loss: 0.2341
Epoch [4/30], Step [300/315], Loss: 0.3502
Epoch [5/30], Step [100/315], Loss: 0.2979
Epoch [5/30], Step [200/315], Loss: 0.3275
Epoch [5/30], Step [300/315], Loss: 0.2028
Epoch [6/30], Step [100/315], Loss: 0.1279
Epoch [6/30], Step [200/315], Loss: 0.2275
Epoch [6/30], Step [300/315], Loss: 0.2324
Epoch [7/30], Step [100/315], Loss: 0.1859
Epoch [7/30], Step [200/315], Loss: 0.1575
Epoch [7/30], Step [300/315], Loss: 0.1972
Epoch [8/30], Step [100/315], Loss: 0.1166
Epoch [8/30], Step [200/315], Loss: 0.1975
Epoch [8/30

In [5]:
for tk in range(num_epochs):
    print('='*40)
    file_name=checkPtFolder+'/'+'epoch_'+str(tk)+'.ckpt'
    state_dict = torch.load(file_name)
    model.load_state_dict(state_dict)
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        trueLabels=[]
        predictedLabels=[]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            trueLabels.extend(labels.tolist())
            predictedLabels.extend(predicted.tolist())
        print(Counter(predictedLabels))
        print(Counter(trueLabels))
        predictedLabels = np.array(predictedLabels)
        trueLabels = np.array(trueLabels)
        for currentClass in range(3):
            print('='*20)
            maskClass=(predictedLabels==currentClass)
            print('class',currentClass,' accuracy_score: ',accuracy_score(predictedLabels[maskClass],trueLabels[maskClass]))
            print('='*20)
        print('f1_weighted',f1_score(trueLabels, predictedLabels, average='weighted'))
        print('f1_macro',f1_score(trueLabels, predictedLabels, average='macro'))
        print('f1_micro',f1_score(trueLabels, predictedLabels, average='micro'))

Counter({2: 5751, 0: 1174, 1: 939})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9633730834752982
class 1  accuracy_score:  0.010649627263045794
class 2  accuracy_score:  0.048861067640410365
f1_weighted 0.2528718133604624
f1_macro 0.12479639882523152
f1_micro 0.18082400813835195
Counter({0: 4846, 2: 2564, 1: 454})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9546017333883615
class 1  accuracy_score:  0.01762114537444934
class 2  accuracy_score:  0.05382215288611544
f1_weighted 0.7174947273264641
f1_macro 0.2928182346083111
f1_micro 0.6068158697863683
Counter({0: 6972, 2: 791, 1: 101})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9519506597819851
class 1  accuracy_score:  0.0297029702970297
class 2  accuracy_score:  0.07332490518331226
f1_weighted 0.8770057220872335
f1_macro 0.35372713774793496
f1_micro 0.8517293997965412
Counter({0: 5183, 2: 2623, 1: 58})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.958904109589041
cl

Counter({0: 7408, 2: 437, 1: 19})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9500539956803455
class 1  accuracy_score:  0.05263157894736842
class 2  accuracy_score:  0.07322654462242563
f1_weighted 0.9015538370178608
f1_macro 0.3524601101011417
f1_micro 0.8991607324516785
Counter({0: 7160, 2: 688, 1: 16})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9506983240223463
class 1  accuracy_score:  0.0
class 2  accuracy_score:  0.07122093023255814
f1_weighted 0.8872575598482262
f1_macro 0.3415973692886094
f1_micro 0.8718209562563581
Counter({0: 7206, 2: 593, 1: 65})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9503191784623924
class 1  accuracy_score:  0.015384615384615385
class 2  accuracy_score:  0.06913996627318718
f1_weighted 0.8895492471686837
f1_macro 0.3459493809040752
f1_micro 0.876144455747711
Counter({0: 7504, 2: 349, 1: 11})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9506929637526652
class 1  accuracy_score:  0